# Formatting Summary Statistics for METAL
- **Project:** GP2 AFR-AAC meta-GWAS 
- **Version:** Python/3.9
- **Status:** COMPLETE
- **Started:** 22-FEB-2023
- **Last Updated:** 22-FEB-2023
    - **Update Description:**  Notebook started

## Notebook Overview
- Format additive summary statistics for meta-GWASes with METAL

### CHANGELOG
- 22-FEB-2023: Notebook started 

---
# Data Overview 

| ANCESTRY |     DATASET     | CASES | CONTROLS |  TOTAL  |           ARRAY           |                NOTES                |
|:--------:|:---------------:|:-----:|:--------:|:-------------------------:|:---------------------------------------------------------------------------------------------------------------:|:-----------------------------------:|
|    AFR   | IPDGC – Nigeria |  304  |    285   |   589   |         NeuroChip         | . | 
|    AFR   |  GP2  |  711  |   1,011  |  1,722  |        NeuroBooster       | . |
|    AAC   |  GP2 |  185  |   1,149  |  1,334  |        NeuroBooster       | . | 
|    AAC   |     23andMe     |  288  |  193,985 | 194,273 | Omni Express & GSA & 550k |        Just summary statistics       |

# Getting Started

## Importing packages

In [19]:
## Import the necessary packages 
import os
import numpy as np
import pandas as pd
import math
import numbers
import sys
import subprocess
import statsmodels.api as sm
import scipy
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

## Print out package versions
## Getting packages loaded into this notebook and their versions to allow for reproducibility
    # Repurposed code from stackoverflow here: https://stackoverflow.com/questions/40428931/package-for-listing-version-of-packages-used-in-a-jupyter-notebook

## Import packages 
import pkg_resources
import types
from datetime import date
today = date.today()
date = today.strftime("%d-%b-%Y").upper()

## Define function 
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            # Split ensures you get root package, not just imported function
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]

        # Some packages are weird and have different imported names vs. system/pip names
        # Unfortunately, there is no systematic way to get pip names from a package's imported name. You'll have to add exceptions to this list manually!
        poorly_named_packages = {
            "PIL": "Pillow",
            "sklearn": "scikit-learn"
        }
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name

## Get a list of packages imported 
imports = list(set(get_imports()))

# The only way I found to get the version of the root package from only the name of the package is to cross-check the names of installed packages vs. imported packages
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

## Print out packages and versions 
print(f"PACKAGE VERSIONS ({date})")
for r in requirements:
    print("\t{}=={}".format(*r))

PACKAGE VERSIONS (22-FEB-2023)
	matplotlib==3.5.2
	numpy==1.22.4
	scipy==1.8.1
	pandas==1.4.3
	statsmodels==0.13.2
	seaborn==0.11.2


# Define Reformatting Function

In [20]:
def format_sumstats_forMETAL_newMarkerIDs(file_path, output_path="test"):
    
    # Read in data from file, assuming the file has a header
    df = pd.read_csv(file_path, sep="\t", low_memory=False)
    
    # Add a new column 'beta' which is the natural log of the 'OR' 
    df['beta'] = df['OR'].apply(np.log)
    
    # Add a new column 'markerID' which is now just the chr:bp:ref:alt
    #df['markerID'] = df['#CHROM'].astype(str) + ':' + df['POS'].astype(str)
    df['markerID'] = df['ID']
    
    # Add new columns 'effect_allele' and 'other_allele'
    df['effect_allele'] = df['A1']
    df['other_allele'] = np.where(df['A1'] == df['REF'], df['ALT'], df['REF'])
    
    # Select the columns of interest and write the result to a file
    output_df = df[['markerID','effect_allele','other_allele','beta', 'LOG(OR)_SE', 'OBS_CT', 'P']]
    output_df.to_csv(f"{output_path}.UpdatedforMETAL.tab", sep='\t', index=False)
    print(f"Saved METAL .tab here: {output_path}.UpdatedforMETAL.tab!")

# Format GWAS summary statistics for METAL 

- [x] IPDGC – Nigeria – AFR - NC
- [x] GP2 release 5 – AFR (public AFR+Nigerian NB)
- [x] GP2 release 5 – AAC

## IPDGC – Nigeria – AFR - NC

In [ ]:
file_path = f"{WORK_DIR}/data/AFR/NIGERIAN-NC/NIGERIAN-NEUROCHIP-AFR-GWAS-MAF005-FEB2023-SUMMARYSTATS.txt"
output_path = f"{WORK_DIR}/data/AFR/NIGERIAN-NC/NIGERIAN-NEUROCHIP-AFR-GWAS-MAF005-FEB2023"

format_sumstats_forMETAL_newMarkerIDs(file_path, output_path)

In [16]:
%%bash

head -5 ${WORK_DIR}/data/AFR/NIGERIAN-NC/NIGERIAN-NEUROCHIP-AFR-GWAS-MAF005-FEB2023.UpdatedforMETAL.tab

markerID	effect_allele	other_allele	beta	LOG(OR)_SE	OBS_CT	P
chr1:722408:G:C	C	G	-0.008473801459398026	0.522089	98	0.98705
chr1:727242:G:A	A	G	-0.49447830558593686	0.419001	158	0.237945
chr1:727717:G:C	C	G	0.8087456094775313	0.500652	101	0.106227
chr1:758351:A:G	G	A	-0.3711854279320762	0.412496	170	0.368199


## GP2 release 4 – AFR (includes public AFR+Nigerian NB)

In [ ]:
file_path = f"{WORK_DIR}/data/AFR/GP2-v5-AFR-wNIGERIAN-NB/GP2-v5-AFR-wNIGERIAN-NB-GWAS-MAF005-FEB2023-SUMMARYSTATS.txt"
output_path = f"{WORK_DIR}/data/AFR/GP2-v5-AFR-wNIGERIAN-NB/GP2-v5-AFR-wNIGERIAN-NB-GWAS-MAF005-FEB2023"

format_sumstats_forMETAL_newMarkerIDs(file_path, output_path)

In [15]:
%%bash

head -5 ${WORK_DIR}/data/AFR/GP2-v5-AFR-wNIGERIAN-NB/GP2-v5-AFR-wNIGERIAN-NB-GWAS-MAF005-FEB2023.UpdatedforMETAL.tab

markerID	effect_allele	other_allele	beta	LOG(OR)_SE	OBS_CT	P
chr1:722408:G:C	C	G	-0.43674062806670166	0.182491	626	0.0167012
chr1:727242:G:A	A	G	-0.36326518336994645	0.180026	844	0.0436073
chr1:727717:G:C	C	G	-0.1961049123092906	0.193128	575	0.309909
chr1:758351:A:G	G	A	-0.4180565486229886	0.178605	917	0.019249


##  GP2 release 4 – AAC

In [ ]:
file_path = f"{WORK_DIR}/data/AAC/GP2-v5-AAC/GP2-v5-AAC-GWAS-MAF005-FEB2023-SUMMARYSTATS.txt"
output_path = f"{WORK_DIR}/data/AAC/GP2-v5-AAC/GP2-v5-AAC-GWAS-MAF005-FEB2023"

format_sumstats_forMETAL_newMarkerIDs(file_path, output_path)

In [14]:
%%bash

head -5 ${WORK_DIR}/data/AAC/GP2-v5-AAC/GP2-v5-AAC-GWAS-MAF005-FEB2023.UpdatedforMETAL.tab

markerID	effect_allele	other_allele	beta	LOG(OR)_SE	OBS_CT	P
chr1:66861:C:T	T	C	-0.10718551328673326	0.400371	580	0.788919
chr1:80346:C:G	G	C	-0.4607854475364722	0.343949	391	0.180346
chr1:595259:G:A	A	G	-0.045603197945461305	0.407711	611	0.91094
chr1:665098:G:A	A	G	-0.03148241119149835	0.357	400	0.92973


## 23andMe Summary Statistics
Processed prior

In [12]:
%%bash

head -5 ${WORK_DIR}/data/23andMe/AAC_23andMe_MAF0.05.hg38.noindels.newMarkerIDs.tab

markerID	effect_allele	alt_allele	effect	stderr	N	pvalue
chr3:25758669:G:A	G	A	1.58859	0.397963	194273	1.74732e-07
chr3:25786115:C:G	G	C	-1.58108	0.397225	194273	1.93941e-07
chr3:25761270:C:T	T	C	-1.58117	0.397615	194273	2.01139e-07
chr3:25764783:G:C	G	C	1.58066	0.397534	194273	2.02106e-07


## chr1 EUR release 4

In [ ]:
file_path = f"{WORK_DIR}/data/other/GP2_v5_EUR_chr1/GP2_v5_EUR_chr1_FEB2023-extractAACAFRhits.txt"
output_path = f"{WORK_DIR}/data/other/GP2_v5_EUR_chr1/GP2_v5_EUR_chr1_extractAACAFRhits-GWAS-MAF005-FEB2023"

format_sumstats_forMETAL_newMarkerIDs(file_path, output_path)